In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
import os
import sys
import datetime
import joblib

In [3]:
# --- 1. Configurações de Ficheiros ---

BASE_DRIVE_PATH = "/content/drive/MyDrive"

TRAINING_DATA_FILE = os.path.join(BASE_DRIVE_PATH, "training_data.csv")
DNABERT_EMBEDDINGS_FILE = os.path.join(BASE_DRIVE_PATH, "dnabert_embeddings.npy")
NORMALIZATION_SCALER_FILE = os.path.join(BASE_DRIVE_PATH, "scaler_standard.pkl")

RESULTS_OUTPUT_DIR = BASE_DRIVE_PATH
os.makedirs(RESULTS_OUTPUT_DIR, exist_ok=True)

RESULTS_FILE_PREFIX = "fnn_optimization_results"
PREDICTIONS_DATA_PREFIX = "fnn_predictions_data"

In [4]:
# --- 2. Configurações da FNN ---
FNN_HIDDEN_DIM = 256
FNN_LEARNING_RATE = 0.001
FNN_NUM_EPOCHS = 100
FNN_BATCH_SIZE = 64

In [5]:
# --- 3. Carregar Dados Originais e Embeddings Pré-Calculados ---

try:
    df = pd.read_csv(TRAINING_DATA_FILE)
    print(f"Dados originais carregados com sucesso de {TRAINING_DATA_FILE}. Dimensão: {df.shape}")
except FileNotFoundError:
    print(f"ERRO: O ficheiro de dados original '{TRAINING_DATA_FILE}' não foi encontrado.")
    sys.exit(1)

if not all(col in df.columns for col in ['target', 'sequence', 'prot_z']):
    print("ERRO: Colunas inesperadas no DataFrame original")
    sys.exit(1)

y_standardized = torch.tensor(df['prot_z'].values, dtype=torch.float32).unsqueeze(1)

scaler = None
try:
    scaler = joblib.load(NORMALIZATION_SCALER_FILE)
    print(f"StandardScaler carregado com sucesso de: {NORMALIZATION_SCALER_FILE}")
    print(f"Média do scaler para desestandardização: {scaler.mean_[0]:.4f}, Desvio Padrão: {scaler.scale_[0]:.4f}")
except FileNotFoundError:
    print(f"ERRO: O ficheiro StandardScaler '{NORMALIZATION_SCALER_FILE}' não foi encontrado.")
    sys.exit(1)
except Exception as e:
    print(f"ERRO: Falha ao carregar StandardScaler de {NORMALIZATION_SCALER_FILE}: {e}")
    sys.exit(1)

true_original_target_values = scaler.inverse_transform(df['prot_z'].values.reshape(-1, 1)).flatten()
print(f"Verdadeiros valores originais do target (para métricas) gerados. Exemplo (primeiros 5): {true_original_target_values[:5]}")

try:
    X_embeddings = np.load(DNABERT_EMBEDDINGS_FILE)
    X_embeddings = torch.tensor(X_embeddings, dtype=torch.float32)
    if X_embeddings.shape[0] != len(df):
        print(f"AVISO: O número de amostras nos embeddings ({X_embeddings.shape[0]}) não corresponde ao número de amostras no CSV ({len(df)}).")
        sys.exit(1)
except FileNotFoundError:
    print(f"ERRO: O ficheiro de embeddings '{DNABERT_EMBEDDINGS_FILE}' não foi encontrado.")
    print(f"Por favor, certifique-se de que o ficheiro .npy com os embeddings está na raiz do seu Google Drive (MyDrive) e acessível.")
    sys.exit(1)
except Exception as e:
    print(f"ERRO: Falha ao carregar o ficheiro de embeddings '{DNABERT_EMBEDDINGS_FILE}': {e}")
    sys.exit(1)

print(f"\nDimensão de X (features): {X_embeddings.shape}")
print(f"Dimensão de y (target 'prot_z' JÁ ESTANDARDIZADO): {y_standardized.shape}")

Dados originais carregados com sucesso de /content/drive/MyDrive/training_data.csv. Dimensão: (9336, 4)


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


StandardScaler carregado com sucesso de: /content/drive/MyDrive/scaler_standard.pkl
Média do scaler para desestandardização: 61077.1584, Desvio Padrão: 64770.3892
Verdadeiros valores originais do target (para métricas) gerados. Exemplo (primeiros 5): [1410.9918352  1443.35117936 1559.28121553 1501.84533491 1680.76889704]

Dimensão de X (features): torch.Size([9336, 768])
Dimensão de y (target 'prot_z' JÁ ESTANDARDIZADO): torch.Size([9336, 1])


In [6]:
# --- 4. Feedforward Neural Network (FNN) ---
class FNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"FNN a treinar em dispositivo: {device}")

FNN a treinar em dispositivo: cpu


In [8]:
# --- 5. Treinar e Avaliar a FNN com Validação Cruzada ---

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

mse_scores_standardized = []
mae_scores_standardized = []
r2_scores_standardized = []
pearson_scores_standardized = []

best_fnn_model_overall = None
best_r2_val = -float('inf')

for fold, (train_index, test_index) in enumerate(kf.split(X_embeddings)):
    print(f"\n--- FNN - Fold {fold+1}/{n_splits} ---")

    X_train, X_test = X_embeddings[train_index], X_embeddings[test_index]
    y_train, y_test = y_standardized[train_index], y_standardized[test_index]

    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=FNN_BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=FNN_BATCH_SIZE, shuffle=False)

    input_dim = X_embeddings.shape[1]
    model = FNN(input_dim, FNN_HIDDEN_DIM).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=FNN_LEARNING_RATE)

    model.train()
    for epoch in range(FNN_NUM_EPOCHS):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if (epoch + 1) % 20 == 0 or epoch == 0 or epoch == FNN_NUM_EPOCHS - 1:
            print(f'  Epoch [{epoch+1}/{FNN_NUM_EPOCHS}], Loss: {total_loss/len(train_loader):.4f}')

    model.eval()
    fold_predictions_standardized = []
    fold_actual_standardized = []
    with torch.no_grad():
        for X_batch_test, y_batch_test in test_loader:
            X_batch_test, y_batch_test = X_batch_test.to(device), y_batch_test.to(device)
            outputs_test = model(X_batch_test)
            fold_predictions_standardized.extend(outputs_test.cpu().numpy().flatten())
            fold_actual_standardized.extend(y_batch_test.cpu().numpy().flatten())

    fold_predictions_standardized = np.array(fold_predictions_standardized)
    fold_actual_standardized = np.array(fold_actual_standardized)

    mse = mean_squared_error(fold_actual_standardized, fold_predictions_standardized)
    mae = mean_absolute_error(fold_actual_standardized, fold_predictions_standardized)
    r2 = r2_score(fold_actual_standardized, fold_predictions_standardized)

    try:
        pearson_corr, _ = pearsonr(fold_actual_standardized, fold_predictions_standardized)
    except ValueError:
        pearson_corr = np.nan

    mse_scores_standardized.append(mse)
    mae_scores_standardized.append(mae)
    r2_scores_standardized.append(r2)
    pearson_scores_standardized.append(pearson_corr)

    print(f"Fold {fold+1} Resultados (escala estandarizada): MSE={mse:.4f}, MAE={mae:.4f}, R2={r2:.4f}, Pearson={pearson_corr:.4f}")

    if r2 > best_r2_val:
        best_r2_val = r2
        best_fnn_model_overall = model.state_dict()

print("\n--- Resultados Finais da Validação Cruzada (Médias e Desvios Padrão) ---")
final_cv_results = {
    "MSE médio (escala estandarizada)": f"{np.nanmean(mse_scores_standardized):.4f} +/- {np.nanstd(mse_scores_standardized):.4f}",
    "MAE médio (escala estandarizada)": f"{np.nanmean(mae_scores_standardized):.4f} +/- {np.nanstd(mae_scores_standardized):.4f}",
    "R2 médio (escala estandarizada)": f"{np.nanmean(r2_scores_standardized):.4f} +/- {np.nanstd(r2_scores_standardized):.4f}",
    "Correlação de Pearson média (escala estandarizada)": f"{np.nanmean(pearson_scores_standardized):.4f} +/- {np.nanstd(pearson_scores_standardized):.4f}"
}
for metric, value in final_cv_results.items():
    print(f"{metric}: {value}")


--- FNN - Fold 1/5 ---
  Epoch [1/100], Loss: 0.7750
  Epoch [20/100], Loss: 0.2511
  Epoch [40/100], Loss: 0.1284
  Epoch [60/100], Loss: 0.0687
  Epoch [80/100], Loss: 0.0354
  Epoch [100/100], Loss: 0.0253
Fold 1 Resultados (escala estandarizada): MSE=0.4469, MAE=0.4920, R2=0.5757, Pearson=0.7680

--- FNN - Fold 2/5 ---
  Epoch [1/100], Loss: 0.8120
  Epoch [20/100], Loss: 0.2616
  Epoch [40/100], Loss: 0.1281
  Epoch [60/100], Loss: 0.0717
  Epoch [80/100], Loss: 0.0394
  Epoch [100/100], Loss: 0.0260
Fold 2 Resultados (escala estandarizada): MSE=0.4480, MAE=0.4884, R2=0.5376, Pearson=0.7682

--- FNN - Fold 3/5 ---
  Epoch [1/100], Loss: 0.8110
  Epoch [20/100], Loss: 0.2727
  Epoch [40/100], Loss: 0.1483
  Epoch [60/100], Loss: 0.0857
  Epoch [80/100], Loss: 0.0502
  Epoch [100/100], Loss: 0.0294
Fold 3 Resultados (escala estandarizada): MSE=0.4303, MAE=0.4823, R2=0.5776, Pearson=0.7801

--- FNN - Fold 4/5 ---
  Epoch [1/100], Loss: 0.8006
  Epoch [20/100], Loss: 0.2571
  Epoch [

In [9]:
# --- 6. Avaliação Final no Dataset Completo e Guardar Dados para Gráficos ---

final_model = FNN(input_dim, FNN_HIDDEN_DIM).to(device)
final_criterion = nn.MSELoss()
final_optimizer = optim.Adam(final_model.parameters(), lr=FNN_LEARNING_RATE)

full_dataset = TensorDataset(X_embeddings, y_standardized)
full_loader = DataLoader(full_dataset, batch_size=FNN_BATCH_SIZE, shuffle=True)

final_model.train()
for epoch in range(FNN_NUM_EPOCHS):
    for X_batch, y_batch in full_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        final_optimizer.zero_grad()
        outputs = final_model(X_batch)
        loss = final_criterion(outputs, y_batch)
        loss.backward()
        final_optimizer.step()

final_model.eval()
final_predictions_standardized = []
with torch.no_grad():
    for X_batch_test, _ in DataLoader(TensorDataset(X_embeddings, y_standardized), batch_size=FNN_BATCH_SIZE, shuffle=False):
        X_batch_test = X_batch_test.to(device)
        outputs_test = final_model(X_batch_test)
        final_predictions_standardized.extend(outputs_test.cpu().numpy().flatten())

final_predictions_standardized = np.array(final_predictions_standardized)

final_predictions_original = scaler.inverse_transform(final_predictions_standardized.reshape(-1, 1)).flatten()

global_mse = mean_squared_error(true_original_target_values, final_predictions_original)
global_mae = mean_absolute_error(true_original_target_values, final_predictions_original)
global_r2 = r2_score(true_original_target_values, final_predictions_original)
try:
    global_pearson, _ = pearsonr(true_original_target_values.flatten(), final_predictions_original.flatten())
except ValueError:
    global_pearson = np.nan

global_results = {
    "MSE Global (escala original)": f"{global_mse:.4f}",
    "MAE Global (escala original)": f"{global_mae:.4f}",
    "R2 Global (escala original)": f"{global_r2:.4f}",
    "Pearson Global (escala original)": f"{global_pearson:.4f}"
}
for metric, value in global_results.items():
    print(f"{metric}: {value}")

MSE Global (escala original): 123223661.1736
MAE Global (escala original): 8545.7397
R2 Global (escala original): 0.9706
Pearson Global (escala original): 0.9856


In [12]:
# --- 7. Guardar Resultados e Modelo ---
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
results_filename = os.path.join(RESULTS_OUTPUT_DIR, f"{RESULTS_FILE_PREFIX}_standard_{timestamp}.txt")
predictions_data_filename = os.path.join(RESULTS_OUTPUT_DIR, f"{PREDICTIONS_DATA_PREFIX}_standard_{timestamp}.npy")
model_save_filename = os.path.join(RESULTS_OUTPUT_DIR, f"best_standard_fnn_model_{timestamp}.pth")

print(f"\n--- A guardar resultados em: {results_filename} ---")
with open(results_filename, 'w') as f:
    f.write(f"Resultados da Otimização da FNN\n")
    f.write(f"Data e Hora: {timestamp}\n")
    f.write(f"----------------------------------------------------\n\n")

    f.write(f"Hiperparâmetros da FNN:\n")
    f.write(f"  Dimensão Oculta: {FNN_HIDDEN_DIM}\n")
    f.write(f"  Taxa de Aprendizagem: {FNN_LEARNING_RATE}\n")
    f.write(f"  Número de Épocas: {FNN_NUM_EPOCHS}\n")
    f.write(f"  Tamanho do Lote: {FNN_BATCH_SIZE}\n")
    f.write(f"\n")

    f.write(f"Resultados da Validação Cruzada (Médias e Desvios Padrão - ESCALA ESTANDARDIZADA):\n")
    for metric, value in final_cv_results.items():
        f.write(f"  {metric}: {value}\n")
    f.write(f"\n")

    f.write(f"Métricas no Dataset Completo (Com o modelo final treinado uma vez no dataset inteiro - ESCALA ORIGINAL):\n")
    for metric, value in global_results.items():
        f.write(f"  {metric}: {value}\n")
    f.write(f"\n")
    f.write(f"Informações Adicionais:\n")
    f.write(f"  Número de Amostras: {X_embeddings.shape[0]}\n")
    f.write(f"  Número de Features: {X_embeddings.shape[1]}\n")
    f.write(f"  Tipo de Embeddings: DNABERT\n")
    f.write(f"  Folds de Validação Cruzada: {n_splits}\n")
    f.write(f"  Ficheiro de Embeddings Usado: {os.path.basename(DNABERT_EMBEDDINGS_FILE)}\n")
    f.write(f"  Ficheiro de Modelo FNN Guardado: {os.path.basename(model_save_filename)}\n")
    f.write(f"  Ficheiro de Dados de Previsões Guardado: {os.path.basename(predictions_data_filename)}\n")
    f.write(f"  Ficheiro de StandardScaler Guardado: {os.path.basename(NORMALIZATION_SCALER_FILE)}\n")


print(f"Resultados guardados com sucesso em {results_filename}.")

data_for_plotting = np.vstack((true_original_target_values.flatten(), final_predictions_original)).T
np.save(predictions_data_filename, data_for_plotting)
print(f"Valores reais (escala original) e previstos (escala original) guardados em {predictions_data_filename} para análise de gráficos.")

torch.save(final_model.state_dict(), model_save_filename)
print(f"Melhor modelo FNN guardado em: {model_save_filename}.")

print("\nScript de treino e avaliação da FNN concluído.")


--- A guardar resultados em: /content/drive/MyDrive/fnn_optimization_results_standard_20250620_230810.txt ---
Resultados guardados com sucesso em /content/drive/MyDrive/fnn_optimization_results_standard_20250620_230810.txt.
Valores reais (escala original) e previstos (escala original) guardados em /content/drive/MyDrive/fnn_predictions_data_standard_20250620_230810.npy para análise de gráficos.
Melhor modelo FNN guardado em: /content/drive/MyDrive/best_standard_fnn_model_20250620_230810.pth.

Script de treino e avaliação da FNN concluído.
